<a href="https://colab.research.google.com/github/vggls/M.Sc._AI_Thesis/blob/main/experiments/breakhis_resnet50/AOPC_Experiments.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **1. Imports**

In [ ]:
pip install grad-cam

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 47.0 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for grad-cam: filename=grad_cam-1.4.6-py3-none-any.whl size=38261 sha256=bd5f0d61c0af255dcd27d91319d60e72406988c6ddead11d7c87b779c1fe0422
  Stored in directory: /root/.cache/pip/wheels/53/02/43/1f75726b5c28459596067ad91e36951463c01273eef661f09f
Successfully built grad-cam


In [ ]:
import plotly.graph_objects as go
import pandas as pd
import numpy as np
import pickle
import shutil

import torch
from torchvision  import transforms

from pytorch_grad_cam import GradCAM, HiResCAM

In [ ]:
# custom written code 
from breakhis_slide_allocation import allocate_slides_to_datasets
from breakhis_binary import create_datasets
from morf import MoRF, AOPC_Dataset, plot_aopc_per_step
from heatmap import Heatmap

In [ ]:
from google.colab import drive  
drive.mount('/content/drive') 

Mounted at /content/drive


# **2. Download from Kaggle**

In [ ]:
! mkdir ~/.kaggle
!cp /content/drive/MyDrive/kaggle.json ~/.kaggle/kaggle.json

In [ ]:
! kaggle datasets download ambarish/breakhis

100% 3.98G/3.99G [00:22<00:00, 83.3MB/s]
100% 3.99G/3.99G [00:22<00:00, 192MB/s] 


In [ ]:
! unzip breakhis.zip

Η έξοδος ροής περικόπηκε στις τελευταίες 5000 γραμμές.
  inflating: BreaKHis_v1/BreaKHis_v1/histology_slides/breast/malignant/SOB/ductal_carcinoma/SOB_M_DC_14-12312/40X/SOB_M_DC-14-12312-40-026.png  
  inflating: BreaKHis_v1/BreaKHis_v1/histology_slides/breast/malignant/SOB/ductal_carcinoma/SOB_M_DC_14-12312/40X/SOB_M_DC-14-12312-40-027.png  
  inflating: BreaKHis_v1/BreaKHis_v1/histology_slides/breast/malignant/SOB/ductal_carcinoma/SOB_M_DC_14-12312/40X/SOB_M_DC-14-12312-40-028.png  
  inflating: BreaKHis_v1/BreaKHis_v1/histology_slides/breast/malignant/SOB/ductal_carcinoma/SOB_M_DC_14-12312/40X/SOB_M_DC-14-12312-40-029.png  
  inflating: BreaKHis_v1/BreaKHis_v1/histology_slides/breast/malignant/SOB/ductal_carcinoma/SOB_M_DC_14-12312/40X/SOB_M_DC-14-12312-40-030.png  
  inflating: BreaKHis_v1/BreaKHis_v1/histology_slides/breast/malignant/SOB/ductal_carcinoma/SOB_M_DC_14-12312/40X/SOB_M_DC-14-12312-40-031.png  
  inflating: BreaKHis_v1/BreaKHis_v1/histology_slides/breast/malignant/SOB/

# **3. Test Dataset**

In [ ]:
#shutil.rmtree('./test_dataset')

In [ ]:
train_transforms = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.RandomHorizontalFlip(p=0.2),
    transforms.RandomVerticalFlip(p=0.2),
    transforms.RandomRotation(20),
    transforms.ToTensor(),
    transforms.Normalize(torch.Tensor([0.5, 0.5, 0.5]), torch.Tensor([0.5, 0.5, 0.5]))
])
# to be applied to validation and test data
test_transforms = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(torch.Tensor([0.5, 0.5, 0.5]), torch.Tensor([0.5, 0.5, 0.5]))
])

In [ ]:
training_dataset, validation_dataset, test_dataset = create_datasets(train_transforms, test_transforms)

No benign slides overlap in training, validation and test sets!
No malignant slides overlap in training, validation and test sets!


In [ ]:
# do not need them in this notebook
shutil.rmtree('./training_dataset')
shutil.rmtree('./validation_dataset')

# **4. Load model**

In [ ]:
model_path = './drive/MyDrive/Colab_Notebooks/dataset_models/BreakHis/Models/'
resnet50 = torch.load(model_path + 'breakhis_resnet50.pt')

In [ ]:
resnet50 = resnet50.cuda()
resnet50 = resnet50.eval()

In [ ]:
layers = [resnet50.conv1, resnet50.bn1, resnet50.layer1, resnet50.layer2, resnet50.layer3, resnet50.layer4]
for child in layers:
    for param in child.parameters():
        param.requires_grad = True

# **5. Experiment function**

In [ ]:
def run_experiment(region_size, labels, xai):

    assert xai in ['gradcam', 'hirescam']

    if xai == 'gradcam':
      cam_instance = GradCAM(model=resnet50, target_layers=[resnet50.layer4[2].conv3], use_cuda=True)
    elif xai == 'hirescam':
      cam_instance = HiResCAM(model=resnet50, target_layers=[resnet50.layer4[2].conv3], use_cuda=True)
    
    print('region_size:{}*{} - labels:{}'.format(region_size, region_size, labels))

    dataset = [(img, label) for (img, label) in test_dataset if (label in labels)]
    diffs, img_scores, aopc_score = AOPC_Dataset(dataset = dataset, 
                                                 model = resnet50, 
                                                 region_size = region_size, 
                                                 cam_instance = cam_instance)

    drive_path = './drive/MyDrive/Colab_Notebooks/dataset_models/BreakHis/AOPC/exp_ResNet50/'

    if xai == 'gradcam':
      with open(drive_path + 'aopc_resnet50_gradcam_diffs_region-size_{}_labels_{}.pickle'.format(region_size, labels), 'wb') as f: pickle.dump(diffs, f)
      with open(drive_path + 'aopc_resnet50_gradcam_scores_region-size_{}_labels_{}.pickle'.format(region_size, labels), 'wb') as f: pickle.dump(img_scores, f)
    elif xai == 'hirescam':
      with open(drive_path + 'aopc_resnet50_hirescam_diffs_region-size_{}_labels_{}.pickle'.format(region_size, labels), 'wb') as f: pickle.dump(diffs, f)
      with open(drive_path + 'aopc_resnet50_hirescam_scores_region-size_{}_labels_{}.pickle'.format(region_size, labels), 'wb') as f: pickle.dump(img_scores, f)
    
    return aopc_score

# **6. GradCAM**

## **region size 16*16** (heatmap 14*14)

In [ ]:
score = run_experiment(region_size=16, labels=[0,1], xai='gradcam')

region_size:16*16 - labels:[0, 1]
Total time: 3010.997667312622 secs
No of correctly classified images: 1316/1557
Avg secs per image:  2.29


In [ ]:
score

0.268

## **region size 21*21** (heatmap 11*11)

In [ ]:
score = run_experiment(region_size=21, labels=[0,1], xai='gradcam')

region_size:21*21 - labels:[0, 1]
Total time: 2011.3037531375885 secs
No of correctly classified images: 1316/1557
Avg secs per image:  1.53


In [ ]:
score

0.276

## **region size 28*28** (heatmap 8*8)

In [ ]:
score = run_experiment(region_size=28, labels=[0,1], xai='gradcam')

region_size:28*28 - labels:[0, 1]
Total time: 1179.170154094696 secs
No of correctly classified images: 1316/1557
Avg secs per image:  0.9


In [ ]:
score

0.27

## **region size 56*56** (heatmap 4*4)

In [ ]:
score = run_experiment(region_size=56, labels=[0,1], xai='gradcam')

region_size:56*56 - labels:[0, 1]
Total time: 420.3759379386902 secs
No of correctly classified images: 1316/1557
Avg secs per image:  0.32


In [ ]:
score

0.324

# **7. HiResCAM**

## **region size 16*16** (heatmap 14*14)

In [ ]:
score = run_experiment(region_size=16, labels=[0,1], xai='hirescam')

region_size:16*16 - labels:[0, 1]
Total time: 3074.348581790924 secs
No of correctly classified images: 1316/1557
Avg secs per image:  2.34


In [ ]:
score

0.266

## **region size 21*21** (heatmap 11*11)

In [ ]:
score = run_experiment(region_size=21, labels=[0,1], xai='hirescam')

region_size:21*21 - labels:[0, 1]
Total time: 2037.9091062545776 secs
No of correctly classified images: 1316/1557
Avg secs per image:  1.55


In [ ]:
score

0.268

## **region size 28*28** (heatmap 8*8)

In [ ]:
score = run_experiment(region_size=28, labels=[0,1], xai='hirescam')

region_size:28*28 - labels:[0, 1]
Total time: 1197.3533585071564 secs
No of correctly classified images: 1316/1557
Avg secs per image:  0.91


In [ ]:
score

0.276

## **region size 56*56** (heatmap 4*4)

In [ ]:
score = run_experiment(region_size=56, labels=[0,1], xai='hirescam')

region_size:56*56 - labels:[0, 1]
Total time: 424.0518753528595 secs
No of correctly classified images: 1316/1557
Avg secs per image:  0.32


In [ ]:
score

0.345